<h1 align="center">PART I</h1>
<h1 align="center">Data Processing and Preparation for Training</h1>

The aim was to process the semantic analysis of English texts, so the collection of positive and negative reviews was taken from [here](http://www.cs.jhu.edu/~mdredze/datasets/sentiment/), this dataset is based on a reviews of various Amazon products.

To get started, install and unzip the archive ([unprocessed.tar.gz](http://www.cs.jhu.edu/~mdredze/datasets/sentiment/unprocessed.tar.gz)) and move the data to the project folder. Now, we import some libraries we need:

In [1]:
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import xml.etree.ElementTree as ET

%matplotlib inline

Also let's write a style for alignment in the middle of all graphs, images, etc:

In [2]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

Since we only need files with positive and negative reviews, let's delete the others. And for our negative and positive reviews, we will add a parent tag in which all the reviews will be, so it will be easier to make xml tree. Also, our xml library will complain about the character `&` when parsing, so we will replace it with the word `and`:

In [3]:
import os

for parent, dirnames, filenames in os.walk('sorted_data'):
    for fn in filenames:
        filepath = os.path.join(parent, fn)
        if not (fn.startswith('positive') or fn.startswith('negative')):
            os.remove(filepath)
        else:
            with open(filepath, 'r') as original: data = original.read().replace("&", "and").replace("", "")
            with open(filepath, 'w') as modified: modified.writelines(["<reviews>\n"] + data.splitlines(True)[1:-1] + ["</reviews>"])

print("Remove unnecessary files!\nAdd additional tag to reviews!")

Remove unnecessary files!
Add additional tag to reviews!


Next, since our reviews are in XML format, we need to process each line to get the value from the tags.

**Note**: 
* **`errors='ignore'`** used to skip all possible incorrect characters in our documents;

* **`readLines`** return all lines in the file, as a list where each line is an item in the list object.

In [4]:
test_read = open('sorted_data/music/positive.review', 'r', encoding='utf8', errors='ignore').readlines()
for i in range(10):
    print(test_read[i])

<reviews>

<review><unique_id>

B00008QS9V:it's_a_mighty_wind_a_blowin'!:carla_j._dinsmore_"acoustic_music_junkie"

</unique_id>

<unique_id>

3276

</unique_id>

<asin>

B00008QS9V

</asin>



In our directory with test training sets we have two files - with positive and negative reviews, each of these files contains 1000 unprocessed reviews. Let's check this:

In [5]:
pos_reviews = open('sorted_data/music/positive.review', 'r', encoding='utf8', errors='ignore').read()
neg_reviews = open('sorted_data/music/negative.review', 'r', encoding='utf8', errors='ignore').read()

pos_rev_tree = ET.fromstring(pos_reviews)
neg_rev_tree = ET.fromstring(neg_reviews)

pos_tags = pos_rev_tree.findall('review')
neg_tags = neg_rev_tree.findall('review')

print('\nNumber of Positive Reviews (Music):', len(pos_tags),
      '\nNumber of Negative Reviews (Music):', len(neg_tags))


Number of Positive Reviews (Music): 1000 
Number of Negative Reviews (Music): 1000


**Note:** 
* **`fromstring()`** parses XML from a string directly into an Element, which is the root element of the parsed tree;
* **`findall() `** finds all elements with a tag which are direct children of the current element (`reviews`).

<h2 align="center">XML Parse</h2>

Now we will try to parse our XML file at a basic level. We will create a dictionary from the lists. Each list will contain certain tags and their content for single review. We already have a tree with positive and negative reviews, so all that remains is to parse it. But before that, let's list all the tags in our xml files so we don't have to repeat:

In [6]:
REVIEW_TAGS = ['unique_id', 'asin', 'product_name', 'helpful', 'rating', 'title',
               'date', 'reviewer', 'reviewer_location', 'review_text']

In [7]:
def parseXML(xml_reviews):
    reviews = {}
    count = 0

    for item in xml_reviews:
        count += 1
        rev_name = 'review' + str(count)
        reviews[rev_name] = [
            REVIEW_TAGS[0] + ' | ' + item.find(REVIEW_TAGS[0]).text.strip(),
            REVIEW_TAGS[1] + ' | ' + item.find(REVIEW_TAGS[1]).text.strip(),
            REVIEW_TAGS[2] + ' | ' + item.find(REVIEW_TAGS[2]).text.strip(),
            REVIEW_TAGS[3] + ' | ' + item.find(REVIEW_TAGS[3]).text.strip(),
            REVIEW_TAGS[4] + ' | ' + item.find(REVIEW_TAGS[4]).text.strip(),
            REVIEW_TAGS[5] + ' | ' + item.find(REVIEW_TAGS[5]).text.strip(),
            REVIEW_TAGS[6] + ' | ' + item.find(REVIEW_TAGS[6]).text.strip(),
            REVIEW_TAGS[7] + ' | ' + item.find(REVIEW_TAGS[7]).text.strip(),
            REVIEW_TAGS[8] + ' | ' + item.find(REVIEW_TAGS[8]).text.strip(),
            REVIEW_TAGS[9] + ' | ' + item.find(REVIEW_TAGS[9]).text.strip()
        ]

    return reviews

Now let's actually create dictionaries with the required values and check output:

In [8]:
pos_reviews_dict = parseXML(pos_tags)
neg_reviews_dict = parseXML(neg_tags)
pos_reviews_dict['review1']

['unique_id | B00008QS9V:it\'s_a_mighty_wind_a_blowin\'!:carla_j._dinsmore_"acoustic_music_junkie"',
 'asin | B00008QS9V',
 'product_name | A Mighty Wind: The Album: Music: Various Artists',
 'helpful | ',
 'rating | 5.0',
 "title | It's a Mighty Wind a blowin'!",
 'date | July 5, 2006',
 'reviewer | Carla J. Dinsmore "Acoustic music junkie"',
 'reviewer_location | Wilmington, DE USA',
 'review_text | This is a wonderful album, that evokes memories of the 60\'s folk boom, yet contains original songs. I was amazed at the fantastic harmonies and musical arrangements.\nAnyone who loves the movie "A Mighty Wind" and who loves folk music will fall in love with this album. I know I did']

Next, we will turn our reviews dictionaries into a Pandas DataFrame:

In [9]:
def dict_to_dataframe(reviews_dict):
    # prepare our dataframe for the data
    df = pd.DataFrame(columns=REVIEW_TAGS)
    count = 0
    for val in reviews_dict.values():
        df.loc[count] = [
            val[0].split("|")[1], val[1].split("|")[1],
            val[2].split("|")[1], val[3].split("|")[1],
            val[4].split("|")[1], val[5].split("|")[1],
            val[6].split("|")[1], val[7].split("|")[1],
            val[8].split("|")[1], val[9].split("|")[1]
        ]

        count = count + 1

    return df

pos_music = dict_to_dataframe(pos_reviews_dict)
neg_music = dict_to_dataframe(neg_reviews_dict)

Let's check for correctness:

In [10]:
pos_music.head(n=3)

,unique_id,asin,product_name,helpful,rating,title,date,reviewer,reviewer_location,review_text
0,B00008QS9V:it's_a_mighty_wind_a_blowin'!:carl...,B00008QS9V,A Mighty Wind: The Album: Music: Various Artists,,5.0,It's a Mighty Wind a blowin'!,"July 5, 2006","Carla J. Dinsmore ""Acoustic music junkie""","Wilmington, DE USA","This is a wonderful album, that evokes memori..."
1,B00005JJ04:sometime_tuesday_morning_defies_de...,B00005JJ04,Sometime Tuesday Morning: Music: Johnny A.,4 of 4,5.0,Sometime Tuesday Morning defies description,"May 3, 2005",Tim Withee,"Auburn, WA United States","On one hand, this CD is a straight ahead inst..."
2,"B0002W4SGS:atreyu_jr:hellrun_""dustin""",B0002W4SGS,The Caitiff Choir: Music: It Dies Today,0 of 1,5.0,atreyu JR,"June 12, 2006","hellrun ""dustin""",wisconsin,this band reminds me of the thrill i first go...


In [11]:
neg_music.head(n=3)

,unique_id,asin,product_name,helpful,rating,title,date,reviewer,reviewer_location,review_text
0,"B00004YWGC:what_can_i_say?:ms._aj_""right""",B00004YWGC,Back For The First Time: Music: Ludacris,1 of 13,2.0,What can I say?,"May 4, 2006","Ms. AJ ""Right""","North Carolina, USA",I've always held the philosophy you are what ...
1,B000621498:not_quite_ready_for_prime_time.:g,B000621498,Things Aren't So Beautiful Now: Music: A Thor...,1 of 2,2.0,not quite ready for prime time.,"May 8, 2006",g,san francisco,someone get this band a producer and put them...
2,"B0000AQS1A:disapointment:super_dave_""super_dave""",B0000AQS1A,Chicken N Beer: Music: Ludacris,1 of 2,2.0,Disapointment,"July 1, 2006","Super Dave ""Super Dave""","Knoxville, TN",Tihs Album is not all that good when it came ...


Everything is correct, wonderful! 

And now we will do what we did above, but for a reviews of all categories of Amazon datasets:

In [12]:
pos_reviews, neg_reviews = [], []
counter = 0 # for styled output 

for parent, dirnames, filenames in os.walk('sorted_data'):
    for fn in filenames:
        filepath = os.path.join(parent, fn)
        if counter % 2 == 0: print("—" * 60)
        
        if fn == "positive.review":
            pos_rev_tree = ET.fromstring(open(filepath, 'r').read())
            pos_tags = pos_rev_tree.findall('review')
            pos_reviews.append(parseXML(pos_tags))
            counter += 1
            print("| Parse", filepath.center(50), "|")
        
        if fn == "negative.review":
            neg_rev_tree = ET.fromstring(open(filepath, 'r').read())
            neg_tags = neg_rev_tree.findall('review')
            neg_reviews.append(parseXML(neg_tags))
            counter += 1
            print("| Parse", filepath.center(50), "|")

print("—" * 60, "\n\n!!! Reviews parsing completed successfully !!!")

————————————————————————————————————————————————————————————
| Parse        sorted_data\apparel\negative.review         |
| Parse        sorted_data\apparel\positive.review         |
————————————————————————————————————————————————————————————
| Parse       sorted_data\automotive\negative.review       |
| Parse       sorted_data\automotive\positive.review       |
————————————————————————————————————————————————————————————
| Parse          sorted_data\baby\negative.review          |
| Parse          sorted_data\baby\positive.review          |
————————————————————————————————————————————————————————————
| Parse         sorted_data\beauty\negative.review         |
| Parse         sorted_data\beauty\positive.review         |
————————————————————————————————————————————————————————————
| Parse         sorted_data\books\negative.review          |
| Parse         sorted_data\books\positive.review          |
————————————————————————————————————————————————————————————
| Parse     sorted_data\

<h3 align="center">Data to Pandas Dataframe</h3>

We now have parsed positive and negative reviews for 25 different categories, so let's check this:

In [13]:
print("Number of Positive Categories:", len(pos_reviews), 
    "\nNumber of Negative Categories:", len(neg_reviews))

print("\nNumber of Positive Reviews (Automotive Category):", len(pos_reviews[1]), 
      "\nNumber of Negative Reviews (Books Category):", len(neg_reviews[4]))

Number of Positive Categories: 25 
Number of Negative Categories: 25

Number of Positive Reviews (Automotive Category): 584 
Number of Negative Reviews (Books Category): 1000


Now, we will turn our reviews dictionaries in each category into a Pandas DataFrame:

**Note:** `pd.concat` - Concatenate pandas objects along a particular axis with optional set logic along the other axes.

In [14]:
positive_df, negative_df = pd.DataFrame(columns=REVIEW_TAGS), pd.DataFrame(columns=REVIEW_TAGS)

In [15]:
for review in pos_reviews:
    print(review['review1'], '\n')
    df = dict_to_dataframe(review)
    positive_df = pd.concat([positive_df, df])
    df.empty

['unique_id | B0007QCQA4:good_sneakers:christopher_w._damico_"macman"', 'asin | B0007QCQA4', "product_name | adidas Originals Men's Superstar II Basketball Shoe: Apparel", 'helpful | 0 of 1', 'rating | 4.0', 'title | GOOD SNEAKERS', 'date | July 15, 2006', 'reviewer | Christopher W. Damico "MACMAN"', 'reviewer_location | NYC', "review_text | GOOD LOOKING KICKS IF YOUR KICKIN IT OLD SCHOOL LIKE ME. AND COMFORTABLE. AND RELATIVELY CHEAP. I'LL ALWAYS KEEP A PAIR OF STAN SMITH'S AROUND FOR WEEKENDS"] 

['unique_id | B0009N4F6I:works_very_well:i_have_9_cats', 'asin | B0009N4F6I', 'product_name | Wedgie Cup Holder: Automotive', 'helpful | 1 of 1', 'rating | 5.0', 'title | Works very well', 'date | August 19, 2006', 'reviewer | I have 9 cats', 'reviewer_location | Dela where?', "review_text | I bought one today for my Mazda Tribute because I didn't like the built-in cup holders. It's actually very sturdy and fits very well."] 

['unique_id | B0002U1SFE:love_it!:a._kurczewski', 'asin | B0002U1

['unique_id | B00008SCFU:fast_and_accurate:j._n._fong', 'asin | B00008SCFU', 'product_name | Kingston 256 MB Secure Digital Card ( SD/256 ): Electronics', 'helpful | ', 'rating | 5.0', 'title | Fast and Accurate', 'date | March 19, 2006', 'reviewer | J. N. Fong', 'reviewer_location | Castro Valley, CA  USA', "review_text | I received my Kingston 256MB SD card just as advertised.\nThe unit came in the mail exactly 2 days after I\nordered. Worked perfectly and I'm very satisfied"] 

['unique_id | B0000TVVEA:an_excellent_italian_gift_basket._.,:stewart__l', 'asin | B0000TVVEA', 'product_name | Italian Gift Basket - Classic (3.8 Pound) by igourmet.com: Gourmet Food', 'helpful | 5 of 11', 'rating | 5.0', 'title | An excellent italian gift basket. .,', 'date | November 24, 2005', 'reviewer | stewart  L', 'reviewer_location | flushing ,ny', 'review_text | This italian gift basket,captures the feeling of italy,at it best...When you send this to family or friends,they will definitly,feel the lo

['unique_id | B00024J7C6:which_is_the_biggest_monster:man_from_u.k._"inside_canon"', 'asin | B00024J7C6', 'product_name | Toilet Monster: Sports andamp; Outdoors', 'helpful | 14 of 22', 'rating | 5.0', 'title | Which is the biggest monster', 'date | January 30, 2006', 'reviewer | Man From U.K. "Inside Canon"', 'reviewer_location | MD USA', "review_text | This monster is great...my old millionaire nan came over recently to tell us that she was about to take us out of her will because we had too much money already, and as luck would have it she had to use the bathroom during that same visit. We heard this scream from the bathroom and when we broke in we found our nan dead on the floor...luckily she hadn't gone to her attorney's office to sign the new will and we got our share after all...thanks toilet monster..you were worth the $15 + $5 shipping we paid for you"] 

['unique_id | B000630AR4:make_life_easier:m._i._guerrero_"sunsad"', 'asin | B000630AR4', 'product_name | Car Wash Wipes: To

In [16]:
for review in neg_reviews:
    df = dict_to_dataframe(review)
    negative_df = pd.concat([negative_df, df])
    df.empty

Let's generate descriptive statistics. Pandas Dataframe descriptive statistics include those that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values. We will also check whether the number of positive and negative reviews has been preserved.

In [17]:
print('\nNumber of Positive Reviews (Dataframe):', len(positive_df),
      '\nNumber of Negative Reviews (Dataframe):', len(negative_df))

positive_df.describe()
negative_df.describe()


Number of Positive Reviews (Dataframe): 21972 
Number of Negative Reviews (Dataframe): 15576


,unique_id,asin,product_name,helpful,rating,title,date,reviewer,reviewer_location,review_text
count,15576,15576,15576,15576,15576,15576,15576,15576,15576,15576
unique,14458,10088,9969,897,2,12487,1662,12536,5472,13758
top,B00005OLX1:sucks_sucks:jason,B00099QAQK,TIME [6-month subscription] [with $5 Bonus]: ...,,1.0,Disappointed,"January 9, 2007",,,This was my first download of this type and a...
freq,4,18,20,2420,9808,72,177,449,3310,18


Actually, `unique_id` has the same information as tags `asin`, `title` and `reviewer` so we can remove it:

In [18]:
positive_df.drop('unique_id', axis=1, inplace=True)
negative_df.drop('unique_id', axis=1, inplace=True)

We will also add a new column `Class`, which will have the value of either `pos` or `neg`, it is necessary to distinguish between positive and negative reviews after concatenation.

In [19]:
positive_df['Class'] = "pos" # positive reviews
negative_df['Class'] = "neg" # negative reviews

reviews = pd.concat([positive_df, negative_df])

Let's see what we got:

In [20]:
reviews.head(n=2)

,asin,product_name,helpful,rating,title,date,reviewer,reviewer_location,review_text,Class
0,B0007QCQA4,adidas Originals Men's Superstar II Basketbal...,0 of 1,4.0,GOOD SNEAKERS,"July 15, 2006","Christopher W. Damico ""MACMAN""",NYC,GOOD LOOKING KICKS IF YOUR KICKIN IT OLD SCHO...,pos
1,B0002Y2JYY,Elite Metal Aviator Sunglasses with Mirror Le...,3 of 5,4.0,Pretty Good,"August 13, 2006","Sharon Civile ""Jackbauerfreak""","Philadelphia, PA",These sunglasses are all right. They were a l...,pos


Let's print the full text of the review (`review_text`) for the text with id 1:

In [22]:
reviews.iloc[0, 8]

" GOOD LOOKING KICKS IF YOUR KICKIN IT OLD SCHOOL LIKE ME. AND COMFORTABLE. AND RELATIVELY CHEAP. I'LL ALWAYS KEEP A PAIR OF STAN SMITH'S AROUND FOR WEEKENDS"

Now our work with the formation of datasets is complete, let's move on.